<a href="https://colab.research.google.com/github/RodrigodeFeo/8DTS/blob/main/An%C3%A1lise_Enem_com_Dados_Completos_completo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import urllib.request
from google.cloud import bigquery
from google.oauth2 import service_account


# Autenticação do Colab com o Bigquery

In [2]:
from google.colab import auth
auth.authenticate_user()

In [5]:
#@title Preenchimento do código do projeto
#@markdown Preencha abaixo o código do teu projeto na GCP. <br/>
#@markdown Após preenchimento do código, execute essa célula.   <br/>

project_id = "intense-wavelet-425818-j0"  #@param {type: "string"}
#@markdown ---
clientbq = bigquery.Client(project=project_id)


A célula a seguir testa a conexão do Big Query, listando todos os datasets disponíveis.

In [6]:
for dataset in clientbq.list_datasets():
  print(dataset.dataset_id)

enem


Teste de conectividade

In [7]:
%%time
query_sql = f"""
  SELECT COUNT( HASHID ) as HASHID, COUNT( DISTINCT HASHID ) as UNIQUE_HASHID
  FROM `{project_id}.enem.enem_all`
"""
query_job = clientbq.query(query_sql)  # Make an API request.
enem_dados = query_job.to_dataframe()
display(enem_dados)

,HASHID,UNIQUE_HASHID
0,16392126,16392126


CPU times: user 84.1 ms, sys: 21 ms, total: 105 ms
Wall time: 4.66 s


## Teste (e template) para busca de dados

Observe na célula a seguir que:
- É declarada uma variável com a instrução SQL a ser executada no cluster de processamento (Big Query);
- É criado um processo de buscas no Big Query;
- Os dados processados são transformado em tabela e armazenado em um DataFrame `enem_dados`.



In [8]:
%%time
query_sql = f"""
  SELECT *
  FROM `{project_id}.enem.enem_all`
  ORDER BY HASHID
  LIMIT 10
"""
query_job = clientbq.query(query_sql)  # Make an API request.
enem_dados = query_job.to_dataframe()
display(enem_dados)

,HASHID,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,...,Q020,Q021,Q022,Q023,Q024,Q025,Q026,Q027,IN_TEMPO_ADICIONAL,TP_FAIXA_ETARIA
0,-9223370068815778474,190001104714,2019,2111300,São Luís,21,MA,27,M,1,...,A,A,B,A,B,B,None,None,0,<NA>
1,-9223368813591070107,200006309895,2020,<NA>,None,<NA>,None,<NA>,F,1,...,A,A,D,A,C,B,None,None,<NA>,2
2,-9223367876145182541,180008452680,2018,3550308,São Paulo,35,SP,17,F,0,...,A,B,E,B,B,B,B,A,<NA>,<NA>
3,-9223367829926564147,200003849740,2020,<NA>,None,<NA>,None,<NA>,F,1,...,None,None,None,None,None,None,None,None,<NA>,14
4,-9223366894718104826,180010649383,2018,2516201,Sousa,25,PB,19,F,0,...,B,A,E,A,B,B,A,A,<NA>,<NA>
5,-9223365105596485985,180010748176,2018,2304285,Eusébio,23,CE,19,F,0,...,A,B,E,A,C,B,A,A,<NA>,<NA>
6,-9223363308800879404,180009470906,2018,2800308,Aracaju,28,SE,18,F,0,...,A,B,E,A,C,B,A,D,<NA>,<NA>
7,-9223362411100011160,190002228980,2019,2410702,Riacho da Cruz,24,RN,26,M,1,...,B,A,C,A,B,B,None,None,0,<NA>
8,-9223362173244671705,180011174377,2018,3550308,São Paulo,35,SP,21,F,0,...,A,B,C,A,A,B,A,A,<NA>,<NA>
9,-9223361115280234371,190002170976,2019,3136207,João Monlevade,31,MG,17,M,1,...,A,A,D,B,A,B,None,None,0,<NA>


CPU times: user 98.9 ms, sys: 13.9 ms, total: 113 ms
Wall time: 4.89 s


# Carregamento dos Dados

<font color='red'> <H1> AQUI COMEÇAM AS DIFERENÇAS </H1> </font>

Não carregamos os dados completos, imagina agora carregar 15 GB de dados neste servidor de aplicação!

Vamos colocar nosso cluster de processamento para funcionar!
Materiais de referência:

- https://cloud.google.com/bigquery/docs/reference/standard-sql/query-syntax
- https://cloud.google.com/bigquery/docs/reference/standard-sql/functions-and-operators


# Questões

## Exemplo 1
Qual a distribuição dos participantes por estado onde a prova foi realizada em cada ano e no total?

### Resposta total

In [9]:
query_sql = f"""
SELECT
  SG_UF_PROVA,
  COUNT( HASHID ) as HASHID
FROM `{project_id}.enem.enem_all`
GROUP BY 1
"""
query_job = clientbq.query(query_sql)  # Make an API request.
enem_dados = query_job.to_dataframe()
display(enem_dados)

,SG_UF_PROVA,HASHID
0,PI,376392
1,DF,319100
2,AP,130521
3,TO,159848
4,SE,243742
5,RS,710906
6,RO,190167
7,ES,320057
8,AL,280353
9,RR,43910


Cria coluna de percentual por estado

In [10]:
# Calcula o percentual sobre o total e armazena em uma nova coluna HASHID_perc
enem_dados["HASHID_perc"] = enem_dados["HASHID"] / enem_dados["HASHID"].sum()

# Muda o formato da nova coluna HASHID_perc para ficar percentual
enem_dados["HASHID_perc"] = enem_dados["HASHID_perc"].map("{:.2%}".format)

#Exibe os resultados
display(enem_dados)

,SG_UF_PROVA,HASHID,HASHID_perc
0,PI,376392,2.30%
1,DF,319100,1.95%
2,AP,130521,0.80%
3,TO,159848,0.98%
4,SE,243742,1.49%
5,RS,710906,4.34%
6,RO,190167,1.16%
7,ES,320057,1.95%
8,AL,280353,1.71%
9,RR,43910,0.27%


In [11]:
px.bar(enem_dados.sort_values("HASHID"),
       y = "HASHID",
       x = "SG_UF_PROVA",
       hover_data=["HASHID", "HASHID_perc"]
       )

In [12]:
px.pie(enem_dados,
       values="HASHID",
       names="SG_UF_PROVA")

### Resposta por ano

In [13]:
query_sql = f"""
SELECT
  SG_UF_PROVA, NU_ANO,
  COUNT( HASHID ) as HASHID
FROM `{project_id}.enem.enem_all`
GROUP BY 1, 2
"""
query_job = clientbq.query(query_sql)  # Make an API request.
enem_dados = query_job.to_dataframe()
display(enem_dados)

,SG_UF_PROVA,NU_ANO,HASHID
0,MS,2020,84548
1,AM,2018,123801
2,SC,2018,125361
3,PA,2020,330883
4,SP,2020,910492
...,...,...,...
76,MS,2018,72393
77,BA,2020,447691
78,ES,2018,111972
79,DF,2019,95860


In [14]:
# Agrupamento por ano
enem_sample_group_ano = enem_dados.groupby(["NU_ANO"]).agg({'HASHID': 'sum'}).reset_index() #Observe a mudança de COUNT para SUM
display(enem_sample_group_ano)

,NU_ANO,HASHID
0,2018,5513747
1,2019,5095270
2,2020,5783109


In [15]:
# JOIN / MERGE das duas tabelas, e já converte o ano para string
enem_sample_final = pd.merge(enem_dados, enem_sample_group_ano, on="NU_ANO", how='left', suffixes = ['', '_ano'])
enem_sample_final['NU_ANO'] = enem_sample_final['NU_ANO'].astype(str)
display(enem_sample_final)

,SG_UF_PROVA,NU_ANO,HASHID,HASHID_ano
0,MS,2020,84548,5783109
1,AM,2018,123801,5513747
2,SC,2018,125361,5513747
3,PA,2020,330883,5783109
4,SP,2020,910492,5783109
...,...,...,...,...
76,MS,2018,72393,5513747
77,BA,2020,447691,5783109
78,ES,2018,111972,5513747
79,DF,2019,95860,5095270


In [16]:
# Calcula o percentual sobre o total e armazena em uma nova coluna HASHID_perc
enem_sample_final["HASHID_perc"]     = enem_sample_final["HASHID"] / enem_dados["HASHID"].sum()
enem_sample_final["HASHID_perc_ano"] = enem_sample_final["HASHID"] / enem_sample_final["HASHID_ano"]

# Muda o formato da nova coluna HASHID_perc para ficar percentual
enem_sample_final["HASHID_perc_ano"] = enem_sample_final["HASHID_perc_ano"].map("{:.2%}".format)
enem_sample_final["HASHID_perc"] = enem_sample_final["HASHID_perc"].map("{:.2%}".format)

#Exibe os resultados
display(enem_sample_final)

,SG_UF_PROVA,NU_ANO,HASHID,HASHID_ano,HASHID_perc,HASHID_perc_ano
0,MS,2020,84548,5783109,0.52%,1.46%
1,AM,2018,123801,5513747,0.76%,2.25%
2,SC,2018,125361,5513747,0.76%,2.27%
3,PA,2020,330883,5783109,2.02%,5.72%
4,SP,2020,910492,5783109,5.55%,15.74%
...,...,...,...,...,...,...
76,MS,2018,72393,5513747,0.44%,1.31%
77,BA,2020,447691,5783109,2.73%,7.74%
78,ES,2018,111972,5513747,0.68%,2.03%
79,DF,2019,95860,5095270,0.58%,1.88%


In [17]:
px.bar(enem_sample_final.sort_values(["NU_ANO", "HASHID"]),
       y = "HASHID",
       x = "SG_UF_PROVA",
       color="NU_ANO",
       barmode='relative',
       hover_data=["HASHID", "HASHID_perc", "HASHID_perc_ano"]
       )

In [18]:
px.bar(enem_sample_final.sort_values(["NU_ANO", "HASHID"]),
       y = "HASHID",
       x = "SG_UF_PROVA",
       color="NU_ANO",
       barmode='group',
       hover_data=["HASHID", "HASHID_perc", "HASHID_perc_ano"]
       )

## Exemplo 2
Considerando apenas 2018, quais as métricas globais de média, mediana, primeiro quartil (25%), terceiro quartil (75%) dos participantes em matemática `NU_NOTA_MT`?


In [19]:
query_sql = f"""
WITH percentiles as
(     SELECT DISTINCT  e.NU_ANO,
         COUNT(HASHID) OVER (PARTITION BY e.NU_ANO) AS HASHID,
        PERCENTILE_DISC(NU_NOTA_MT, 0.03) OVER(PARTITION BY e.NU_ANO) AS p03,
        PERCENTILE_DISC(NU_NOTA_MT, 0.25) OVER(PARTITION BY e.NU_ANO) AS q1,
        PERCENTILE_DISC(NU_NOTA_MT, 0.50) OVER(PARTITION BY e.NU_ANO) AS median,
        PERCENTILE_DISC(NU_NOTA_MT, 0.75) OVER(PARTITION BY e.NU_ANO) AS q3,
        PERCENTILE_DISC(NU_NOTA_MT, 0.97) OVER(PARTITION BY e.NU_ANO) AS p97
      FROM `{project_id}.enem.enem_all` AS e
      WHERE e.NU_ANO = 2018
      AND e.TP_PRESENCA_MT = 1 )

SELECT e.NU_ANO,
      min(e.NU_NOTA_MT) as min,
      avg(e.NU_NOTA_MT) as mean,
      max(e.NU_NOTA_MT) as max,
      STDDEV(e.NU_NOTA_MT) as std,
      p.HASHID as count,
      p.p03,
      p.q1,
      p.median,
      p.q3,
      p.p97
FROM  `{project_id}.enem.enem_all` AS e
JOIN percentiles as p
ON p.NU_ANO = e.NU_ANO
GROUP BY e.NU_ANO,
      p.HASHID,
      p.p03,
      p.q1,
      p.median,
      p.q3,
      p.p97
"""
query_job = clientbq.query(query_sql)  # Make an API request.
enem_dados_agg = query_job.to_dataframe()
display(enem_dados_agg)

,NU_ANO,min,mean,max,std,count,p03,q1,median,q3,p97
0,2018,0.0,535.405566,996.1,103.151243,3905099,388.6,455.3,516.6,600.7,752.5


In [20]:
fig = go.Figure()
fig.add_trace(go.Box(orientation="h"))
fig.update_traces(q1=[enem_dados_agg.loc[0,"q1"]],
                  median=[enem_dados_agg.loc[0,"median"]],
                  q3=[enem_dados_agg.loc[0,"q3"]],
                  lowerfence=[enem_dados_agg.loc[0,"min"]],
                  upperfence=[enem_dados_agg.loc[0,"max"]], )

fig.show()

## Exemplo 3
Faça a mesma análise, porém considerando quebras por estado (local da prova).
Indique quais estados tem métricas superiores e inferiores do que as métricas nacionais.


In [21]:
query_sql = f"""
WITH percentiles as
(      SELECT DISTINCT  e.NU_ANO, e.SG_UF_PROVA,
         COUNT(HASHID) OVER (PARTITION BY e.NU_ANO, e.SG_UF_PROVA) AS HASHID,
        PERCENTILE_DISC(NU_NOTA_MT, 0.03) OVER(PARTITION BY e.NU_ANO, e.SG_UF_PROVA) AS p03,
        PERCENTILE_DISC(NU_NOTA_MT, 0.25) OVER(PARTITION BY e.NU_ANO, e.SG_UF_PROVA) AS q1,
        PERCENTILE_DISC(NU_NOTA_MT, 0.50) OVER(PARTITION BY e.NU_ANO, e.SG_UF_PROVA) AS median,
        PERCENTILE_DISC(NU_NOTA_MT, 0.75) OVER(PARTITION BY e.NU_ANO, e.SG_UF_PROVA) AS q3,
        PERCENTILE_DISC(NU_NOTA_MT, 0.97) OVER(PARTITION BY e.NU_ANO, e.SG_UF_PROVA) AS p97
      FROM `{project_id}.enem.enem_all` AS e
      WHERE e.NU_ANO = 2018
      AND e.TP_PRESENCA_MT = 1
)

SELECT e.NU_ANO, e.SG_UF_PROVA,
      min(e.NU_NOTA_MT) as min,
      avg(e.NU_NOTA_MT) as mean,
      max(e.NU_NOTA_MT) as max,
      STDDEV(e.NU_NOTA_MT) as std,
      p.HASHID as count,
      p.p03,
      p.q1,
      p.median,
      p.q3,
      p.p97
FROM  `{project_id}.enem.enem_all` AS e
JOIN percentiles as p
ON p.NU_ANO = e.NU_ANO
AND p.SG_UF_PROVA = e.SG_UF_PROVA
GROUP BY e.NU_ANO, e.SG_UF_PROVA,
      p.HASHID,
      p.p03,
      p.q1,
      p.median,
      p.q3,
      p.p97
"""
query_job = clientbq.query(query_sql)  # Make an API request.
enem_dados_estado = query_job.to_dataframe()
enem_dados_estado.index = enem_dados_estado["SG_UF_PROVA"]
display(enem_dados_estado)

,NU_ANO,SG_UF_PROVA,min,mean,max,std,count,p03,q1,median,q3,p97
SG_UF_PROVA,,,,,,,,,,,,
DF,2018,DF,0.0,545.004771,996.1,109.755360,74431,388.8,459.6,525.2,618.6,771.3
RR,2018,RR,0.0,514.910864,921.0,91.997726,9223,386.4,446.2,498.4,566.6,722.0
MT,2018,MT,0.0,518.060743,986.4,95.383072,63704,386.2,445.4,500.4,573.2,728.5
SE,2018,SE,0.0,519.117620,985.6,98.696947,57496,385.2,443.9,499.6,575.0,738.9
CE,2018,CE,0.0,529.160650,996.1,102.921982,232617,387.4,450.7,508.5,590.3,751.9
SP,2018,SP,0.0,559.202934,996.1,107.487023,643149,395.2,474.7,543.2,636.9,771.9
AL,2018,AL,0.0,516.464331,962.0,95.212347,65521,385.7,444.1,498.5,570.6,728.2
RS,2018,RS,0.0,544.500579,989.5,101.487416,167702,391.7,465.2,529.8,614.0,749.5
PI,2018,PI,0.0,512.597112,996.1,98.326702,93621,384.8,439.1,491.3,564.4,737.5


In [22]:
fig = go.Figure()
boxplot_data = {"SG_UF_PROVA":["Brasil"],
                "p03": [enem_dados_agg.loc[0,"p03"]],
                "q1": [enem_dados_agg.loc[0,"q1"]],
                "median": [enem_dados_agg.loc[0,"median"]],
                "q3": [enem_dados_agg.loc[0,"q3"]],
                "p97": [enem_dados_agg.loc[0,"p97"]],
                "lowerfence": [enem_dados_agg.loc[0,"min"]],
                "upperfence": [enem_dados_agg.loc[0,"max"]],
                }

for estado in enem_dados_estado.sort_values(["median"])["SG_UF_PROVA"]:
  mydata = enem_dados_estado.loc[estado, :]

  boxplot_data["SG_UF_PROVA"].append(estado)
  boxplot_data["q1"].append(mydata["q1"])
  boxplot_data["median"].append(mydata["median"])
  boxplot_data["q3"].append(mydata["q3"])
  boxplot_data["lowerfence"].append(mydata["min"]) # Experimente com p03
  boxplot_data["upperfence"].append(mydata["max"]) # Experimente com p97

fig.add_trace(go.Box())
fig.update_traces(x=boxplot_data["SG_UF_PROVA"],
                  q1=boxplot_data["q1"],
                  median=boxplot_data["median"],
                  q3=boxplot_data["q3"],
                  lowerfence=boxplot_data["lowerfence"],
                  upperfence=boxplot_data["upperfence"])
fig.update_layout(height=650)
fig.show()

# ATIVIDADES

<font color="red">Acompanhar slide com enunciados</font>

## Exercício 1
Faça a mesma análise em 2018 realizada por estado, porém observe como a escolaridade da mãe influencia nas notas do ENEM (Q002). Comente sua análise!

In [31]:
query_sql = f"""
SELECT
  SG_UF_PROVA, NU_ANO, Q002, AVG(NU_NOTA_MT) as media
FROM `{project_id}.enem.enem_all`
WHERE NU_ANO = 2018
GROUP BY 1, 2, 3
ORDER BY Q002
"""
query_job = clientbq.query(query_sql)  # Make an API request.
enem_dados = query_job.to_dataframe()
display(enem_dados)

,SG_UF_PROVA,NU_ANO,Q002,media
0,AP,2018,A,472.073228
1,PB,2018,A,478.182746
2,TO,2018,A,468.706236
3,RJ,2018,A,485.105882
4,SE,2018,A,475.707888
...,...,...,...,...
211,SP,2018,H,525.313488
212,TO,2018,H,486.750352
213,RR,2018,H,498.405498
214,PR,2018,H,520.522586


In [38]:
# Calcula o percentual sobre o total e armazena em uma nova coluna HASHID_perc
enem_dados["MEDIA_perc"]     = enem_dados["media"] / enem_dados["media"].sum()
enem_dados["MEDIA_perc_ano"] = enem_dados["media"] / enem_dados["NU_ANO"]

# Muda o formato da nova coluna HASHID_perc para ficar percentual
enem_dados["MEDIA_perc_ano"] = enem_dados["MEDIA_perc_ano"].map("{:.2%}".format)
enem_dados["MEDIA_perc"] = enem_dados["MEDIA_perc"].map("{:.2%}".format)

#Exibe os resultados
display(enem_dados)

,SG_UF_PROVA,NU_ANO,Q002,media,MEDIA_perc,MEDIA_perc_ano
0,AP,2018,A,472.073228,0.42%,23.39%
1,PB,2018,A,478.182746,0.42%,23.70%
2,TO,2018,A,468.706236,0.42%,23.23%
3,RJ,2018,A,485.105882,0.43%,24.04%
4,SE,2018,A,475.707888,0.42%,23.57%
...,...,...,...,...,...,...
211,SP,2018,H,525.313488,0.47%,26.03%
212,TO,2018,H,486.750352,0.43%,24.12%
213,RR,2018,H,498.405498,0.44%,24.70%
214,PR,2018,H,520.522586,0.46%,25.79%


In [50]:
query_sql = f"""
WITH percentiles as
(      SELECT DISTINCT  e.NU_ANO, e.Q002,
         COUNT(HASHID) OVER (PARTITION BY e.NU_ANO, e.Q002) AS HASHID,
        PERCENTILE_DISC(NU_NOTA_MT, 0.03) OVER(PARTITION BY e.NU_ANO, e.Q002) AS p03,
        PERCENTILE_DISC(NU_NOTA_MT, 0.25) OVER(PARTITION BY e.NU_ANO, e.Q002) AS q1,
        PERCENTILE_DISC(NU_NOTA_MT, 0.50) OVER(PARTITION BY e.NU_ANO, e.Q002) AS median,
        PERCENTILE_DISC(NU_NOTA_MT, 0.75) OVER(PARTITION BY e.NU_ANO, e.Q002) AS q3,
        PERCENTILE_DISC(NU_NOTA_MT, 0.97) OVER(PARTITION BY e.NU_ANO, e.Q002) AS p97
      FROM `{project_id}.enem.enem_all` AS e
      WHERE e.NU_ANO = 2018
      AND e.TP_PRESENCA_MT = 1
)

SELECT e.NU_ANO, e.Q002,
      min(e.NU_NOTA_MT) as min,
      avg(e.NU_NOTA_MT) as mean,
      max(e.NU_NOTA_MT) as max,
      STDDEV(e.NU_NOTA_MT) as std,
      p.HASHID as count,
      p.p03,
      p.q1,
      p.median,
      p.q3,
      p.p97
FROM  `{project_id}.enem.enem_all` AS e
JOIN percentiles as p
ON p.NU_ANO = e.NU_ANO
AND p.Q002 = e.Q002
GROUP BY e.NU_ANO, e.Q002,
      p.HASHID,
      p.p03,
      p.q1,
      p.median,
      p.q3,
      p.p97
"""
query_job = clientbq.query(query_sql)  # Make an API request.
enem_dados = query_job.to_dataframe()
enem_dados.index = enem_dados["Q002"]
display(enem_dados)

,NU_ANO,Q002,min,mean,max,std,count,p03,q1,median,q3,p97
Q002,,,,,,,,,,,,
C,2018,C,0.0,511.969966,994.4,87.539283,517552,386.3,445.3,498.0,564.7,703.5
F,2018,F,0.0,591.357692,996.1,116.236232,418303,401.7,496.2,583.0,686.4,800.6
B,2018,B,0.0,496.919650,996.1,80.800814,608759,384.0,435.9,483.7,544.5,678.9
D,2018,D,0.0,521.219833,996.1,90.833547,511009,388.2,451.9,507.0,576.5,716.8
H,2018,H,0.0,503.910272,993.0,90.103055,110651,383.7,437.2,486.9,553.4,710.2
E,2018,E,0.0,541.930710,996.1,100.121808,1278305,391.8,464.0,526.4,608.7,746.2
A,2018,A,0.0,478.144011,982.5,73.717795,136928,381.2,422.8,464.4,519.8,648.2
G,2018,G,0.0,604.586684,996.1,118.263307,323592,405.3,506.8,603.2,700.5,810.9


In [51]:
fig = go.Figure()
boxplot_data = {"Q002":["Brasil"],
                "p03": [enem_dados_agg.loc[0,"p03"]],
                "q1": [enem_dados_agg.loc[0,"q1"]],
                "median": [enem_dados_agg.loc[0,"median"]],
                "q3": [enem_dados_agg.loc[0,"q3"]],
                "p97": [enem_dados_agg.loc[0,"p97"]],
                "lowerfence": [enem_dados_agg.loc[0,"min"]],
                "upperfence": [enem_dados_agg.loc[0,"max"]],
                }

for qq2 in enem_dados.sort_values(["median"])["Q002"]:
  mydata = enem_dados.loc[qq2, :]

  boxplot_data["Q002"].append(qq2)
  boxplot_data["q1"].append(mydata["q1"])
  boxplot_data["median"].append(mydata["median"])
  boxplot_data["q3"].append(mydata["q3"])
  boxplot_data["lowerfence"].append(mydata["min"]) # Experimente com p03
  boxplot_data["upperfence"].append(mydata["max"]) # Experimente com p97

fig.add_trace(go.Box())
fig.update_traces(x=boxplot_data["Q002"],
                  q1=boxplot_data["q1"],
                  median=boxplot_data["median"],
                  q3=boxplot_data["q3"],
                  lowerfence=boxplot_data["lowerfence"],
                  upperfence=boxplot_data["upperfence"])
fig.update_layout(height=650)
fig.show()

Percebemos que quanto maior a escolaridade da mae maior a media da nota!

## Exercício 2
Faça a mesma análise em 2018 realizada por estado, porém observe como ter computador em casa influencia nas notas do ENEM (Q024). Comente sua análise!


In [52]:
query_sql = f"""
WITH percentiles as
(      SELECT DISTINCT  e.NU_ANO, e.Q024,
         COUNT(HASHID) OVER (PARTITION BY e.NU_ANO, e.Q024) AS HASHID,
        PERCENTILE_DISC(NU_NOTA_MT, 0.03) OVER(PARTITION BY e.NU_ANO, e.Q024) AS p03,
        PERCENTILE_DISC(NU_NOTA_MT, 0.25) OVER(PARTITION BY e.NU_ANO, e.Q024) AS q1,
        PERCENTILE_DISC(NU_NOTA_MT, 0.50) OVER(PARTITION BY e.NU_ANO, e.Q024) AS median,
        PERCENTILE_DISC(NU_NOTA_MT, 0.75) OVER(PARTITION BY e.NU_ANO, e.Q024) AS q3,
        PERCENTILE_DISC(NU_NOTA_MT, 0.97) OVER(PARTITION BY e.NU_ANO, e.Q024) AS p97
      FROM `{project_id}.enem.enem_all` AS e
      WHERE e.NU_ANO = 2018
      AND e.TP_PRESENCA_MT = 1
)

SELECT e.NU_ANO, e.Q024,
      min(e.NU_NOTA_MT) as min,
      avg(e.NU_NOTA_MT) as mean,
      max(e.NU_NOTA_MT) as max,
      STDDEV(e.NU_NOTA_MT) as std,
      p.HASHID as count,
      p.p03,
      p.q1,
      p.median,
      p.q3,
      p.p97
FROM  `{project_id}.enem.enem_all` AS e
JOIN percentiles as p
ON p.NU_ANO = e.NU_ANO
AND p.Q024 = e.Q024
GROUP BY e.NU_ANO, e.Q024,
      p.HASHID,
      p.p03,
      p.q1,
      p.median,
      p.q3,
      p.p97
"""
query_job = clientbq.query(query_sql)  # Make an API request.
enem_dados = query_job.to_dataframe()
enem_dados.index = enem_dados["Q024"]
display(enem_dados)

,NU_ANO,Q024,min,mean,max,std,count,p03,q1,median,q3,p97
Q024,,,,,,,,,,,,
A,2018,A,0.0,498.781807,996.1,81.106154,1554125,384.4,437.8,485.9,546.2,681.7
B,2018,B,0.0,545.056074,996.1,101.673222,1863619,391.9,465.4,529.8,614.9,749.9
D,2018,D,0.0,636.451398,996.1,116.906388,102624,420.3,543.8,653.5,724.6,829.7
E,2018,E,0.0,660.795488,996.1,116.932148,43598,431.5,575.6,683.4,743.4,851.5
C,2018,C,0.0,603.110983,996.1,114.964828,341133,407.6,509.3,602.2,695.6,804.2


In [53]:
fig = go.Figure()
boxplot_data = {"Q024":["Brasil"],
                "p03": [enem_dados_agg.loc[0,"p03"]],
                "q1": [enem_dados_agg.loc[0,"q1"]],
                "median": [enem_dados_agg.loc[0,"median"]],
                "q3": [enem_dados_agg.loc[0,"q3"]],
                "p97": [enem_dados_agg.loc[0,"p97"]],
                "lowerfence": [enem_dados_agg.loc[0,"min"]],
                "upperfence": [enem_dados_agg.loc[0,"max"]],
                }

for qq2 in enem_dados.sort_values(["median"])["Q024"]:
  mydata = enem_dados.loc[qq2, :]

  boxplot_data["Q024"].append(qq2)
  boxplot_data["q1"].append(mydata["q1"])
  boxplot_data["median"].append(mydata["median"])
  boxplot_data["q3"].append(mydata["q3"])
  boxplot_data["lowerfence"].append(mydata["min"]) # Experimente com p03
  boxplot_data["upperfence"].append(mydata["max"]) # Experimente com p97

fig.add_trace(go.Box())
fig.update_traces(x=boxplot_data["Q024"],
                  q1=boxplot_data["q1"],
                  median=boxplot_data["median"],
                  q3=boxplot_data["q3"],
                  lowerfence=boxplot_data["lowerfence"],
                  upperfence=boxplot_data["upperfence"])
fig.update_layout(height=650)
fig.show()

## Exercício 3
Faça a mesma análise em 2018 realizada por estado, porém observe como a renda mensal da família influencia nas notas do ENEM (Q006). Comente sua análise!


In [54]:
query_sql = f"""
WITH percentiles as
(      SELECT DISTINCT  e.NU_ANO, e.Q006,
         COUNT(HASHID) OVER (PARTITION BY e.NU_ANO, e.Q006) AS HASHID,
        PERCENTILE_DISC(NU_NOTA_MT, 0.03) OVER(PARTITION BY e.NU_ANO, e.Q006) AS p03,
        PERCENTILE_DISC(NU_NOTA_MT, 0.25) OVER(PARTITION BY e.NU_ANO, e.Q006) AS q1,
        PERCENTILE_DISC(NU_NOTA_MT, 0.50) OVER(PARTITION BY e.NU_ANO, e.Q006) AS median,
        PERCENTILE_DISC(NU_NOTA_MT, 0.75) OVER(PARTITION BY e.NU_ANO, e.Q006) AS q3,
        PERCENTILE_DISC(NU_NOTA_MT, 0.97) OVER(PARTITION BY e.NU_ANO, e.Q006) AS p97
      FROM `{project_id}.enem.enem_all` AS e
      WHERE e.NU_ANO = 2018
      AND e.TP_PRESENCA_MT = 1
)

SELECT e.NU_ANO, e.Q006,
      min(e.NU_NOTA_MT) as min,
      avg(e.NU_NOTA_MT) as mean,
      max(e.NU_NOTA_MT) as max,
      STDDEV(e.NU_NOTA_MT) as std,
      p.HASHID as count,
      p.p03,
      p.q1,
      p.median,
      p.q3,
      p.p97
FROM  `{project_id}.enem.enem_all` AS e
JOIN percentiles as p
ON p.NU_ANO = e.NU_ANO
AND p.Q006 = e.Q006
GROUP BY e.NU_ANO, e.Q006,
      p.HASHID,
      p.p03,
      p.q1,
      p.median,
      p.q3,
      p.p97
"""
query_job = clientbq.query(query_sql)  # Make an API request.
enem_dados = query_job.to_dataframe()
enem_dados.index = enem_dados["Q006"]
display(enem_dados)

,NU_ANO,Q006,min,mean,max,std,count,p03,q1,median,q3,p97
Q006,,,,,,,,,,,,
L,2018,L,0.0,629.273960,994.4,113.881915,42177,420.2,538.9,644.3,716.5,817.3
Q,2018,Q,0.0,682.368340,996.1,113.377709,45231,440.6,614.8,703.8,758.4,866.1
F,2018,F,0.0,557.879003,987.9,101.810371,175734,396.1,477.8,546.1,633.4,753.4
I,2018,I,0.0,596.672619,996.1,111.101932,138771,407.7,506.9,593.9,686.6,791.2
H,2018,H,0.0,584.114093,996.1,108.032258,168064,403.8,497.4,577.2,671.2,777.8
G,2018,G,0.0,565.188668,996.1,103.320102,276659,398.7,483.5,554.2,644.0,760.3
A,2018,A,0.0,488.114601,996.1,78.358858,153939,382.6,430.3,474.3,531.2,671.0
M,2018,M,0.0,633.268215,996.1,114.417374,37405,422.0,543.2,649.0,719.8,821.6
B,2018,B,0.0,493.373645,988.5,78.190923,894418,383.6,434.8,480.9,539.1,670.8


In [55]:
fig = go.Figure()
boxplot_data = {"Q006":["Brasil"],
                "p03": [enem_dados_agg.loc[0,"p03"]],
                "q1": [enem_dados_agg.loc[0,"q1"]],
                "median": [enem_dados_agg.loc[0,"median"]],
                "q3": [enem_dados_agg.loc[0,"q3"]],
                "p97": [enem_dados_agg.loc[0,"p97"]],
                "lowerfence": [enem_dados_agg.loc[0,"min"]],
                "upperfence": [enem_dados_agg.loc[0,"max"]],
                }

for qq2 in enem_dados.sort_values(["median"])["Q006"]:
  mydata = enem_dados.loc[qq2, :]

  boxplot_data["Q006"].append(qq2)
  boxplot_data["q1"].append(mydata["q1"])
  boxplot_data["median"].append(mydata["median"])
  boxplot_data["q3"].append(mydata["q3"])
  boxplot_data["lowerfence"].append(mydata["min"]) # Experimente com p03
  boxplot_data["upperfence"].append(mydata["max"]) # Experimente com p97

fig.add_trace(go.Box())
fig.update_traces(x=boxplot_data["Q006"],
                  q1=boxplot_data["q1"],
                  median=boxplot_data["median"],
                  q3=boxplot_data["q3"],
                  lowerfence=boxplot_data["lowerfence"],
                  upperfence=boxplot_data["upperfence"])
fig.update_layout(height=650)
fig.show()

# DESAFIO

Seção reservada para os corajosos de espírito

Considere que a nota final do ENEM é composta pela nota das quatro áreas mais a nota de redação, responda:

Como a escolaridade de ambos os pais SIMULTANEAMENTE influencia na nota final do ENEM?


Dica: Primeramente examine cada um dos pais individualmente, então faça uma nova análise considerando todo o casal (A+A, A+B, B+A, etc.)



CN – Ciências da Natureza
CH – Ciências Humanas
LC – Linguagens e Códigos
MT – Matemática
REDAÇÃO